In [1]:
# Bibliotetecas para tratamento de dados
import pandas as pd
import numpy as np
from haversine import haversine

# bibliotecas para apresentação gráfica

import plotly.express as px
import plotly.graph_objects as go
import folium





In [2]:
df = pd.read_csv('train.csv')
dataset = df.copy()

In [3]:
dataset.head()

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weatherconditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken(min)
0,0x4607,INDORES13DEL02,37,4.9,22.745049,75.892471,22.765049,75.912471,19-03-2022,11:30:00,11:45:00,conditions Sunny,High,2,Snack,motorcycle,0,No,Urban,(min) 24
1,0xb379,BANGRES18DEL02,34,4.5,12.913041,77.683237,13.043041,77.813237,25-03-2022,19:45:00,19:50:00,conditions Stormy,Jam,2,Snack,scooter,1,No,Metropolitian,(min) 33
2,0x5d6d,BANGRES19DEL01,23,4.4,12.914264,77.678400,12.924264,77.688400,19-03-2022,08:30:00,08:45:00,conditions Sandstorms,Low,0,Drinks,motorcycle,1,No,Urban,(min) 26
3,0x7a6a,COIMBRES13DEL02,38,4.7,11.003669,76.976494,11.053669,77.026494,05-04-2022,18:00:00,18:10:00,conditions Sunny,Medium,0,Buffet,motorcycle,1,No,Metropolitian,(min) 21
4,0x70a2,CHENRES12DEL01,32,4.6,12.972793,80.249982,13.012793,80.289982,26-03-2022,13:30:00,13:45:00,conditions Cloudy,High,1,Snack,scooter,1,No,Metropolitian,(min) 30


# Limpeza de dados

In [4]:
#retirando NaN

filtro =( (dataset['Delivery_person_Age'] != 'NaN ')  & (dataset['multiple_deliveries'] != 'NaN ') 
& (dataset['Road_traffic_density'] != 'NaN ') & 
(dataset['Type_of_order'] != 'NaN ') & (dataset['City'] != 'NaN ')& (dataset['Festival'] != 'NaN ') )
dataset = dataset.loc[filtro, :]
# Convertendo tipo dos dados
dataset['Delivery_person_Age'] = dataset['Delivery_person_Age'].astype(int)
dataset['Delivery_person_Ratings'] = dataset['Delivery_person_Ratings'].astype(float)
dataset['multiple_deliveries'] = dataset['multiple_deliveries'].astype(int)

#transformação de data
dataset['Order_Date'] = pd.to_datetime(dataset['Order_Date'], format='%d-%m-%Y')

#transformando time_taken()
dataset['Time_taken(min)'] = dataset['Time_taken(min)'].apply(lambda x: x.split()[1])
dataset['Time_taken(min)'] = dataset['Time_taken(min)'].astype(int)

#retirando espaços
dataset['Festival'] = dataset['Festival'].str.strip()
dataset['ID'] = dataset['ID'].str.strip()
dataset['Weatherconditions'] = dataset['Weatherconditions'].str.strip()
dataset['Road_traffic_density'] = dataset['Road_traffic_density'].str.strip()
dataset['Type_of_order'] = dataset['Type_of_order'].str.strip()
dataset['Type_of_vehicle'] = dataset['Type_of_vehicle'].str.strip()
dataset['City'] = dataset['City'].str.strip()



In [5]:
dataset.dtypes



ID                                     object
Delivery_person_ID                     object
Delivery_person_Age                     int32
Delivery_person_Ratings               float64
Restaurant_latitude                   float64
Restaurant_longitude                  float64
Delivery_location_latitude            float64
Delivery_location_longitude           float64
Order_Date                     datetime64[ns]
Time_Orderd                            object
Time_Order_picked                      object
Weatherconditions                      object
Road_traffic_density                   object
Vehicle_condition                       int64
Type_of_order                          object
Type_of_vehicle                        object
multiple_deliveries                     int32
Festival                               object
City                                   object
Time_taken(min)                         int32
dtype: object

# 1.0 Visão Empresa

1. Quantidade de pedidos por dia.

In [6]:
df1 = dataset.loc[: , ['ID', 'Order_Date']].groupby(['Order_Date']).count().reset_index()
df1

px.bar(df1, x='Order_Date', y='ID')


2. Quantidade de pedidos por semana.


In [7]:
dataset['week'] = dataset['Order_Date'].dt.strftime('%U')
df1 = dataset.loc[:, ['ID', 'week']].groupby(['week']).count().reset_index()
px.line(df1, x='week', y='ID')

3. Distribuição dos pedidos por tipo de tráfego.

In [8]:
df1 = dataset.loc[:, ['ID', 'Road_traffic_density']].groupby(['Road_traffic_density']).count().reset_index()

df1['%']= df1['ID'] / df1['ID'].sum()

px.pie(df1, values='%', names='Road_traffic_density')



4. Comparação do volume de pedidos por cidade e tipo de tráfego.


In [9]:
df1 = dataset.loc[:, ['ID', 'City', 'Road_traffic_density']].groupby(['City', 'Road_traffic_density']).count().reset_index()
df1

px.scatter(df1, x='City', y='Road_traffic_density', size='ID', color = 'City')

5. A quantidade de pedidos por entregador por semana.


In [10]:
df1 = dataset.loc[:, ['ID', 'week']].groupby(['week']).count().reset_index()
df2 = dataset.loc[:,['Delivery_person_ID','week']].groupby(['week']).nunique().reset_index()

df_aux = pd.merge(df1, df2, how='inner')
df_aux['%'] = df_aux['ID'] / df_aux['Delivery_person_ID']
px.line(df_aux, x='week', y='%')

6. A localização central de cada cidade por tipo de tráfego.

In [11]:
df.columns
cols = ['Restaurant_latitude','Restaurant_longitude','Delivery_location_latitude','Delivery_location_longitude','Road_traffic_density', 'City']
df1 = dataset.loc[:, cols].groupby(['City','Road_traffic_density']).mean().reset_index()
map = folium.Map()

for index, location in df1.iterrows():
    folium.Marker([location['Restaurant_latitude'], 
                  location['Restaurant_longitude']],
                  popup=location[['City','Road_traffic_density']]).add_to(map)

map


# 2. Visão Entregadores

1. A menor e maior idade dos entregadores.


In [12]:
age_max = dataset['Delivery_person_Age'].max()
age_min = dataset['Delivery_person_Age'].min()

age_max, age_min

(39, 20)

2. A pior e a melhor condição de veículos.


In [13]:
worst_condition = dataset['Vehicle_condition'].min()
best_condition = dataset['Vehicle_condition'].max()

best_condition, worst_condition

(2, 0)

3. A avaliação médida por entregador.


In [14]:
df1 = dataset.loc[:,['Delivery_person_Ratings', 'Delivery_person_ID']].groupby(['Delivery_person_ID']).mean().reset_index()
df1

,Delivery_person_ID,Delivery_person_Ratings
0,AGRRES010DEL01,4.761538
1,AGRRES010DEL02,4.671429
2,AGRRES010DEL03,4.575000
3,AGRRES01DEL01,4.522222
4,AGRRES01DEL02,4.700000
...,...,...
1315,VADRES19DEL02,4.632727
1316,VADRES19DEL03,4.670270
1317,VADRES20DEL01,4.620370
1318,VADRES20DEL02,4.591111


4. A avaliação média e o desvio padrão por tipo de tráfego.


In [15]:
df1 = (dataset.loc[:, ['Delivery_person_Ratings', 'Road_traffic_density']].groupby([ 'Road_traffic_density'])
                                                                            .agg({'Delivery_person_Ratings': ['mean', 'std']}))
df1.columns = ['avg_traffic', 'std_traffic']
df1 = df1.reset_index()
df1

,Road_traffic_density,avg_traffic,std_traffic
0,High,4.652230,0.273044
1,Jam,4.594019,0.329778
2,Low,4.645011,0.338080
3,Medium,4.660138,0.274245


5. A avaliação média e o desvio padrão por condições climáticas.


In [16]:
df1 = dataset.loc[:, ['Delivery_person_Ratings', 'Weatherconditions']].groupby(['Weatherconditions']).agg({'Delivery_person_Ratings': ['mean', 'std']})

df1.columns = ['avg_conditions', 'std_conditions']

df1 =df1.reset_index()
df1


,Weatherconditions,avg_conditions,std_conditions
0,conditions Cloudy,4.651871,0.281197
1,conditions Fog,4.652965,0.275060
2,conditions Sandstorms,4.611748,0.310852
3,conditions Stormy,4.611819,0.313096
4,conditions Sunny,4.654868,0.396674
5,conditions Windy,4.616128,0.304565


6. Os 10 entregadores mais rápidos por cidade.


In [17]:
df1 = dataset.loc[:, ['Time_taken(min)', 'Delivery_person_ID','City']].groupby(['Delivery_person_ID','City']).mean().sort_values('Time_taken(min)', ascending=True).reset_index().head(10)
df1

,Delivery_person_ID,City,Time_taken(min)
0,BHPRES11DEL03,Urban,10.000000
1,DEHRES08DEL02,Urban,10.000000
2,KOCRES04DEL02,Urban,10.000000
3,DEHRES20DEL03,Urban,10.000000
4,AURGRES02DEL03,Urban,10.000000
5,LUDHRES01DEL02,Urban,10.000000
6,BHPRES13DEL01,Urban,11.000000
7,LUDHRES07DEL03,Urban,11.000000
8,KOLRES05DEL01,Urban,11.000000
9,CHENRES07DEL01,Urban,11.666667


7. Os 10 entregadores mais lentos por cidade

In [18]:
df1 = dataset.loc[:, ['Time_taken(min)', 'Delivery_person_ID','City']].groupby(['Delivery_person_ID','City']).mean().sort_values('Time_taken(min)', ascending=False).reset_index().head(10)
df1

,Delivery_person_ID,City,Time_taken(min)
0,PUNERES13DEL02,Semi-Urban,54.0
1,MYSRES12DEL01,Semi-Urban,54.0
2,AGRRES17DEL01,Semi-Urban,54.0
3,JAPRES14DEL02,Semi-Urban,54.0
4,JAPRES09DEL01,Semi-Urban,54.0
5,VADRES13DEL01,Semi-Urban,54.0
6,JAPRES03DEL02,Semi-Urban,54.0
7,BANGRES010DEL01,Semi-Urban,54.0
8,LUDHRES05DEL01,Semi-Urban,54.0
9,MYSRES04DEL03,Semi-Urban,54.0


# 3. Visão Restaurantes

1. A quantidade de entregadores únicos.

In [19]:
df1 = dataset['Delivery_person_ID'].nunique()
df1

1320

2. A distância média dos resturantes em relação aos locais de entrega.


In [20]:
cols = ['Restaurant_latitude','Restaurant_longitude', 'Delivery_location_latitude', 'Delivery_location_longitude', 'City']
df1 = dataset.loc[:, cols]
df1['distance(km)'] = df1.loc[:, cols].apply(lambda x: np.round(haversine(
                                    (x['Restaurant_latitude'], x['Restaurant_longitude'])
                                    ,(x['Delivery_location_latitude'], x['Delivery_location_longitude'])
                                    ),2), axis=1
                                    )



In [21]:
avg_distance = df1.loc[:, ['distance(km)', 'City']].groupby(['City']).mean().reset_index()




In [30]:
px.pie(avg_distance, values='distance(km)', names='City', width=600)

3. O tempo médio e o desvio padrão de entrega por cidade.


In [23]:
df1 = dataset.loc[:, ['Time_taken(min)', 'City']].groupby(['City']).agg({'Time_taken(min)': ['mean', 'std']})
df1.columns = ['avg_time_delivery', 'std_time_delivery']
df1 = df1.reset_index()

fig = go.Figure()
fig.add_trace(go.Bar(
    name='Control',
    x=df1['City'],
    y=df1['avg_time_delivery'],
    error_y = dict(type='data', array=df1['std_time_delivery']
                   )

)
              )

fig.update_layout(barmode='group')
fig.show()


4. O tempo médio e o desvio padrão de entrega por cidade e tipo de pedido.


In [24]:
df1 = dataset.loc[: , ['Time_taken(min)', 'City','Type_of_order']].groupby(['City', 'Type_of_order']).agg({'Time_taken(min)': ['mean', 'std']})

df1.columns = ['avg_time', 'std_time']
df1 = df1.reset_index()
df1

,City,Type_of_order,avg_time,std_time
0,Metropolitian,Buffet,27.299008,9.153107
1,Metropolitian,Drinks,27.322691,9.041655
2,Metropolitian,Meal,27.616383,9.214536
3,Metropolitian,Snack,27.468414,9.119676
4,Semi-Urban,Buffet,49.707317,2.731702
5,Semi-Urban,Drinks,49.625000,2.459347
6,Semi-Urban,Meal,50.300000,3.041665
7,Semi-Urban,Snack,49.408163,2.707385
8,Urban,Buffet,23.560652,9.056348
9,Urban,Drinks,23.311977,8.927314


5. O tempo médio e o desvio padrão de entrega por cidade e tipo de tráfego.


In [29]:
df1 = dataset.loc[: , ['Time_taken(min)', 'City','Road_traffic_density']].groupby(['City', 'Road_traffic_density']).agg({'Time_taken(min)': ['mean', 'std']})

df1.columns = ['avg_time', 'std_time']
df1 = df1.reset_index()

fig = px.sunburst(df1, path=['City', 'Road_traffic_density'], values='avg_time',
                  color='std_time',width=600, color_continuous_scale='RdBu', color_continuous_midpoint=np.average(df1['std_time']))

fig.show()

c:\Users\nrodr\anaconda3\lib\site-packages\plotly\express\_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

c:\Users\nrodr\anaconda3\lib\site-packages\plotly\express\_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



6. O tempo médio de entrega durantes os Festivais.

In [26]:
df1 = dataset.loc[:, ['Time_taken(min)', 'Festival']].groupby(['Festival']).agg({'Time_taken(min)': ['mean', 'std']})
df1.columns = ['avg_time', 'std_time']
df1 = df1.reset_index()
df1


,Festival,avg_time,std_time
0,No,26.162741,9.001803
1,Yes,45.518607,4.005399


In [27]:
df1['Festival'].unique()

array(['No', 'Yes'], dtype=object)

In [28]:
# Ao ter festival

yes_festival = df1.loc[df1['Festival'] == 'Yes', 'avg_time']
yes_festival
# Sem festival
no_festival = df1.loc[df1['Festival'] == 'No', 'avg_time']
no_festival

0    26.162741
Name: avg_time, dtype: float64